In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing  Contractions library for fixing contractions for e.g.
"I'm" is a contraction of "I am."
"you're" is a contraction of "you are."
"can't" is a contraction of "cannot."

In [47]:
pip install contractions

In [48]:
# Importing necessary libaries.
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re , string, unicodedata
import contractions
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import os





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
#Reading the input.xlsl file using pandas and storing it in a variable 'df'.

df=pd.read_excel(r"/content/drive/MyDrive/NLP assignment/20211030 Test Assignment/Input.xlsx")

In [50]:
#Displaying the first 5 rows of the data.

df.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [51]:
#Checking the number of rows present in the dataframe.

len(df['URL'])

100

In [52]:
#Creating an empty list of data.
data=[]

# Iterate over each row in the DataFrame 'df'
for index, row in df.iterrows():
  #Extract 'URL_ID' and 'URL' from the current row
  URL_ID=row['URL_ID']
  URL=row['URL']

  # Send an HTTP request to the URL and check if the response status code is 200 (OK)
  page=requests.get(URL)
  if(page.status_code==200):
    print("Data fetched succesfully",URL)

    # Parse the HTML content of the page using BeautifulSoup
    soup=BeautifulSoup(page.content,'html.parser')

    # Extract the article title from the HTML using specific classes
    Article_Title=soup.findAll('h1',class_='entry-title') + soup.findAll('h1',class_='tdb-title-text')
    Article_Title=Article_Title[0].text

    # Removing the References present at the end of each article under 'pre' elements with class 'wp-block-preformatted'
    for preformatted_element in soup.find_all('pre', class_='wp-block-preformatted'):
      preformatted_element.decompose()

    # Extract article text candidates using specific classes
    Article_Text_candidates=soup.findAll(attrs={'class' : "td-post-content tagdiv-type"}) + soup.findAll(attrs={'class':'tdb-block-inner td-fix-index'})

    # The URL link 14 has its content under class 'tdb-block-inner td-fix-index' in 14th index.
    if len(Article_Text_candidates)>14:
      Article_Text=Article_Text_candidates[14].text.replace('\n',' ')

    # For rest of URL where article is present in '0th; index.
    elif Article_Text_candidates:
      Article_Text=Article_Text_candidates[0].text.replace('\n',' ')
    else:
      Article_Text="Text not available"

    # Append the extracted data to the empty 'data' list that was created above.
    data.append([URL_ID,URL,Article_Title,Article_Text])

  #If URL is not found display this statement.
  else:
    print("URL not found",URL)


Data fetched succesfully https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/
Data fetched succesfully https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/
Data fetched succesfully https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/
Data fetched succesfully https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/
Data fetched succesfully https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/
Data fetched succesfully https://insights.blackcoffer.com/the-rise-of-the-ott-platform-and-its-impact-on-the-entertainment-industry-by-2040/
Data fetched succesfully https://insights.blackcoffer.com/rise-of-cyber-crime-and-its-effects/
Data fetched succesfully https://insights.blackcoffer.com/rise

*There are two unavailable URL's found.*

In [53]:
data=pd.DataFrame(data,columns=['URL_ID','URL','Article_title','Article_text'])

In [54]:
#pd.set_option('display.max_colwidth',None)
data.head()

,URL_ID,URL,Article_title,Article_text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,We have seen a huge development and dependenc...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"Throughout history, from the industrial revol..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...",Introduction In the span of just a few decade...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"The way we live, work, and communicate has un..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,The year 2040 is poised to witness a continue...


In [55]:
# Checking the overall information of data.

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL_ID         98 non-null     object
 1   URL            98 non-null     object
 2   Article_title  98 non-null     object
 3   Article_text   98 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [56]:
# Defining a function named 'replace_contractions' that takes text as input.
def replace_contractions(text):
  """Replace contractions in a piece of texts"""

  # Using the contractions library to fix contractions in the input text
  return(contractions.fix(text))

# Applying the replace_contractions function to each element in the 'Article_text' column of the 'data' DataFrame
data['Article_text']=data['Article_text'].apply(lambda x: replace_contractions(x))

In [57]:
# Defining a function named remove_numbers that takes a text as input

def remove_numbers(text):

  # Using the re.sub() function to replace all sequences of digits (\d+) with an empty string in the input text
  text=re.sub(r'\d+','',text)

  # Returning the modified text without numbers
  return text

# Applying the remove_numbers function to each element in the 'Article_text' column of the 'data' DataFrame
data['Article_text']=data['Article_text'].apply(lambda x: remove_numbers(x))

In [58]:
# This line uses the re.findall() method to find all occurrences of the specified patterns in the input text (row)
# \b\w+\b: This matches whole words. \b denotes a word boundary, and \w+ matches one or more word characters (letters, digits, or underscores).
# [.,;?!]: This matches any of the specified punctuation characters (., ;, !, ?).
# (?<=\w)\.(?=\w): This matches periods that are not part of an abbreviation.
# It uses positive lookbehind (?<=\w) to ensure there is a word character before the period,
# and positive lookahead (?=\w) to ensure there is a word character after the period.

data['Article_text']=data['Article_text'].apply(lambda row: re.findall(r'\b\w+\b|[.,;?!]|(?<=\w)\.(?=\w)', row))


In [59]:
#Applying a Lambda function to tokenize each word and punctuation present in the data and join them into a single string, separating each element with a space.

data['Article_text'] = data['Article_text'].apply(lambda row: word_tokenize(' '.join(row)))

In [60]:
data.head()

,URL_ID,URL,Article_title,Article_text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"[We, have, seen, a, huge, development, and, de..."
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"[Throughout, history, ,, from, the, industrial..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","[Introduction, In, the, span, of, just, a, few..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"[The, way, we, live, ,, work, ,, and, communic..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,"[The, year, is, poised, to, witness, a, contin..."


In [61]:
# Keeping a copy of the transformed data in 5 variables so that i can use each for different calculations.

data_1=data.copy()
data_2=data.copy()
data_3=data.copy()
data_4=data.copy()
data_5=data.copy()


# Creating custom stop words

In [62]:
# Providing the path of custom stopwords.
custom_stopwords_file_path=r"/content/drive/MyDrive/NLP assignment/20211030 Test Assignment/StopWords"

In [63]:
# Define a function named load_stopwords that takes a folder path as input
def load_stopwords(stopwords_folder):
  # Create an empty set to store stopwords
  stopwords=set()

  #Iterate over files in the stopwords folder.
  for filename in os.listdir(stopwords_folder):

    # Construct the full path to the file
    file_path=os.path.join(stopwords_folder,filename)

    #Read stopwords from the file and add them to the set.
    with open(file_path,'r',encoding='latin-1') as file:

      # Update the set with stopwords from the current file
      stopwords.update(word.strip() for word in file)

  # Return the set of stopwords
  return stopwords


In [64]:
# Load English stopwords from NLTK library
stopwords=stopwords.words('english')

# Print the length of the initial set of stopword
print("Length of NLTK Stopwords: ",len(stopwords))

# Load custom stopwords from a specified file path using the load_stopwords function
custom_stopwords=list(load_stopwords(custom_stopwords_file_path))

# Convert all custom stopwords to lowercase
custom_stopwords=[words.lower() for words in custom_stopwords]

# Combine the initial set of stopwords and the custom stopwords
stopwords=list(stopwords + custom_stopwords)

# Print the length of the combined set of stopwords
print("Length of combined Stopwords: ",len(stopwords))



Length of NLTK Stopwords:  179
Length of combined Stopwords:  13098


In [65]:
# Functions to normalize a list of tokenized words (convert to lowercase and remove stopwords)

def to_lowercase(words):
  """Converts all characters to lower case from the list of tokenized words"""
  new_words=[]
  for word in words:
    new_word=word.lower()
    new_words.append(new_word)
  return new_words


def remove_stopwords(words):
  """Removes Stopwords from the list of tokenized words"""
  new_words=[]
  for word in words:
    if word not in stopwords:
      new_words.append(word)
  return new_words

def normalize(words):
  words=to_lowercase(words)
  words=remove_stopwords(words)
  return ' '.join(words)

# Apply the normalize function to each element in the 'Article_text' column of the 'data' DataFrame
data['Article_text']=data['Article_text'].apply(lambda row: normalize(row))

# Tokenize the normalized text using the word_tokenize function
data['Article_text'] = data['Article_text'].apply(lambda row: word_tokenize(row))

# Display the first few rows of the updated DataFrame
data.head()



,URL_ID,URL,Article_title,Article_text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"[huge, development, dependence, people, techno..."
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"[history, ,, industrial, revolution, century, ..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","[introduction, span, decades, ,, internet, und..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"[live, ,, work, ,, communicate, unquestionably..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,"[poised, witness, continued, revolution, world..."


In [66]:
# Defining a function that will read from the 'master dictionary' folder and load the words into a dictionary with key 'Positive' for positive words and 'Negative' for neagtive words.

def create_sentiment_dictionary(master_dict_folder,stopwords):

  # Initialize sets to store positive and negative word
  positive_words=set()
  negative_words=set()

  #Iterate over files in the master dictionary.
  for sentiment in ['positive','negative']:

    # Construct the file path for the current sentiment
    filename=f"{sentiment}-words.txt"
    file_path=os.path.join(master_dict_folder,filename)

    #Read words from the file and add them to the positive or negative set.
    with open(file_path,'r',encoding='latin-1') as file:

      # Create a list of words, stripping leading and trailing whitespaces, excluding stopwords.
      words=[word.strip() for word in file if word.strip() not in stopwords]

      # Update the corresponding set based on sentiment
      if sentiment=='positive':
        positive_words.update(words)
      elif sentiment=='negative':
        negative_words.update(words)

  #Create a dictionary with positive and negative words.
  sentiment_dictionary={'positive':positive_words,'negative':negative_words}
  return sentiment_dictionary


In [67]:
# Providing the path of MasterDictionary Folder.
master_dictionary_folder=r"/content/drive/MyDrive/NLP assignment/20211030 Test Assignment/MasterDictionary"

In [68]:
# Calling the function 'create_sentiment_dictionary' and storing in the variable named 'sentiment_dictionary.

sentiment_dictionary=create_sentiment_dictionary(master_dictionary_folder,stopwords)

In [69]:
# Extract the set of positive words from the sentiment_dictionary
positive_words=sentiment_dictionary['positive']

In [70]:
# Extract the set of negative words from the sentiment_dictionary
negative_words=sentiment_dictionary['negative']

In [71]:
# This line defines a function named derive_variable that takes a list of tokenized words (tokenized_words), a set of positive words (positive_dict), and a set of negative words (negative_dict) as input.

def derive_variable(tokenized_words,positive_dict,negative_dict):
  #Initialize score
  positive_score=0
  negative_score=0
  #Iterate through tokens and calculate positive or negative scores.
  for token in tokenized_words:
    if token in positive_dict:
      positive_score+=1
    elif token in negative_dict:
      negative_score-=1

  # Multiply negative score by -1 to make it positive
  negative_score=-1*negative_score

  return positive_score,negative_score

# Apply the derive_variable function to each element in the 'Article_text' column of the 'data' DataFrame
data[['Positive_Score','Negative_Score']]=data['Article_text'].apply(lambda row: pd.Series(derive_variable(row,positive_words,negative_words),index=['Positive_Score','Negative_Score']))

data[['Positive_Score','Negative_Score']].head()

,Positive_Score,Negative_Score
0,27,6
1,52,31
2,36,25
3,35,74
4,21,9


In [72]:
# Calculate the Polarity Score for each row

data['Polarity_Score']=(data['Positive_Score']-data['Negative_Score'])/((data['Positive_Score']+data['Negative_Score'])+ 0.000001)

data["Polarity_Score"].head()

0    0.636364
1    0.253012
2    0.180328
3   -0.357798
4    0.400000
Name: Polarity_Score, dtype: float64

In [73]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

# Function to convert all characters to lowercase from a list of tokenized words
def to_lowercase(words):
  """Converts all characters to lower case from the list of tokenized words"""
  new_words=[]
  for word in words:
    new_word=word.lower()
    new_words.append(new_word)
  return new_words

# Function to remove punctuations from the list of tokenized words
def remove_punctuations(words):
  """Removes punctuations from the list of tokenized words"""
  new_words=[]
  for word in words:
    # Remove punctuations using a regular expression
    new_word=re.sub(r'[^\w\s]','',word)

    # Add the non-empty word to the list
    if new_word != '':
      new_words.append(new_word)
  return new_words

# Function to remove stopwords from the list of tokenized words
def remove_stopwords(words):
  """Removes Stopwords from the list of tokenized words"""
  new_words=[]
  for word in words:

    # Check if the word is not in the stopwords list and add it to the new list
    if word not in stopwords:
      new_words.append(word)
  return new_words

# Function to normalize a list of tokenized words
def normalize(words):
  # Apply text preprocessing steps
  words=to_lowercase(words)
  words=remove_punctuations(words)
  words=remove_stopwords(words)
  # Join the remaining words into a single string
  return ' '.join(words)

# Apply the normalize function to each element in the 'Article_text' column of the 'data_1' DataFrame
data_1['Article_text']=data_1['Article_text'].apply(lambda row: normalize(row))

# Tokenize the normalized text using the nltk.word_tokenize function
data_1['Article_text']=data_1['Article_text'].apply(lambda row: nltk.word_tokenize(row))

# Calculate the total number of words after cleaning and assign it to the 'Word_Count' column
Total_words_after_cleaning=data_1['Article_text'].apply(lambda row:len(row))

data['Word_Count']=Total_words_after_cleaning
data['Word_Count']

0     613
1     868
2     674
3     657
4     413
     ... 
93    652
94    517
95    245
96    336
97    594
Name: Word_Count, Length: 98, dtype: int64

In [74]:
# Calculate the Subjectivity Score for each row
data['Subjectivity_Score']=(data['Positive_Score']+ data['Negative_Score'])/((Total_words_after_cleaning)+0.000001)

data['Subjectivity_Score'].head()

0    0.053834
1    0.095622
2    0.090504
3    0.165906
4    0.072639
Name: Subjectivity_Score, dtype: float64

# 2.	Analysis of Readability

In [75]:
#Function to count syllables in a word.
def count_syllables(word):
  # Check for special cases where 'es' or 'ed' at the end does not contribute an extra syllable
  if word.endswith('es') or word.endswith('ed'):
    return max(1, sum([1 for char in word[:-2] if char in 'aeiou']))
  else:
    return max(1,sum([1 for char in word if char in 'aeiou']))

#Function to count complex words in a text.

def count_complex_words(words,syllable_threshold=2):
  # Count the number of words with syllables greater than the specified threshold
  complex_words_count=sum(1 for word in words if count_syllables(word) > syllable_threshold)

  return complex_words_count

# Function to convert all characters to lowercase from a list of tokenized words
def to_lowercase(words):
  """Converts all characters to lower case from the list of tokenized words"""
  new_words=[]
  for word in words:
    new_word=word.lower()
    new_words.append(new_word)
  return new_words

# Function to remove punctuations from the list of tokenized words
def remove_punctuations(words):
  """Removes punctuations from the list of tokenized words"""
  new_words=[]
  for word in words:
    new_word=re.sub(r'[^\w\s]','',word)
    if new_word != '':
      new_words.append(new_word)
  return new_words

# Function to remove stopwords from the list of tokenized words
def remove_stopwords(words):
  """Removes Stopwords from the list of tokenized words"""
  new_words=[]
  for word in words:
    if word not in stopwords:
      new_words.append(word)
  return new_words

# Function to normalize a list of tokenized words
def normalize(words):
  words=to_lowercase(words)
  words=remove_stopwords(words)
  words=remove_punctuations(words)
  return ' '.join(words)

# Apply the normalize function to each element in the 'Article_text' column of the 'data_2' DataFrame
data_2['Article_text']=data_2["Article_text"].apply(lambda row: normalize(row))

# Tokenize the normalized text using the nltk.word_tokenize function
data_2['Article_text']=data_2['Article_text'].apply(lambda row: nltk.word_tokenize(row))

#Calculate the number of syllables per word and assign it to the 'Syllable_Per_Word' column
data['Syllable_Per_Word']=data_2['Article_text'].apply(lambda row: [count_syllables(word) for word in row])

# Calculate the count of complex words and assign it to the 'Complex_Word_Count' column
data['Complex_Word_Count']=data_2['Article_text'].apply(lambda row: count_complex_words(row))


In [76]:
# Function to convert all characters to lowercase from a list of tokenized sentences
def to_lowercase(sentences):
  """Converts all characters to lower case from the list of tokenized words"""
  new_sentences=[]
  for sentence in sentences:
    new_sentence=sentence.lower()
    new_sentences.append(new_sentence)
  return " ".join(new_sentences)

# Apply the to_lowercase function to each element in the 'Article_text' column of the 'data_3' DataFrame
data_3['Article_text']=data_3['Article_text'].apply(lambda row: to_lowercase(row))

# Tokenize the normalized text into sentences using the nltk.sent_tokenize function
data_3['sentence_per_row']=data_3['Article_text'].apply(lambda row: nltk.sent_tokenize(row))

# Calculate the number of sentences per row and assign it to the 'number_of_sentences' column in the 'data' DataFrame
data['number_of_sentences']=data_3['sentence_per_row'].apply(len)
data['number_of_sentences'].head()

0    78
1    81
2    56
3    51
4    39
Name: number_of_sentences, dtype: int64

In [77]:
# Function to remove punctuations from a list of tokenized words
def remove_punctuations(words):
  """Removes punctuations from the list of tokenized words"""
  new_words=[]
  for word in words:
    new_word=re.sub(r'[^\w\s]','',word)
    if new_word != '':
      new_words.append(new_word)
  return ' '.join(new_words)

# Apply the remove_punctuations function to each element in the 'Article_text' column of the 'data_5' DataFrame
data_5['Article_text']=data_5['Article_text'].apply(lambda row: remove_punctuations(row))

# Tokenize the cleaned text using the nltk.word_tokenize function
data_5['Article_text']=data_5['Article_text'].apply(lambda row: nltk.word_tokenize(row))

# Calculate the number of words per row and assign it to the 'number_of_words' column in the 'data' DataFrame
data['number_of_words']=data_5['Article_text'].apply(lambda row:len(row))
data['number_of_words'].head()


0    1194
1    1464
2    1072
3    1043
4     688
Name: number_of_words, dtype: int64

In [78]:
# Function to calculate the average word length in a list of words
def average_word_length(words):
  total_characters=sum(len(word) for word in words)
  total_words=len(words)
  return total_characters/total_words

# Apply the average_word_length function to each element in the 'Article_text' column of the 'data_5' DataFrame
data['Avg_Word_Length']=data_5['Article_text'].apply(average_word_length)
data['Avg_Word_Length'].head()


0    4.564489
1    5.396858
2    5.992537
3    5.915628
4    5.386628
Name: Avg_Word_Length, dtype: float64

In [79]:
# Calculate the average sentence length for each row
data['Average_Sentence_Length']= data['number_of_words']/data['number_of_sentences']
data['Average_Sentence_Length'].head()

0    15.307692
1    18.074074
2    19.142857
3    20.450980
4    17.641026
Name: Average_Sentence_Length, dtype: float64

In [80]:
# Calculate the percentage of complex words for each row
data['Percentage_Of_Complex_Words']=data['Complex_Word_Count']/data['number_of_words']
data['Percentage_Of_Complex_Words'].head()

0    0.190955
1    0.283470
2    0.350746
3    0.346117
4    0.258721
Name: Percentage_Of_Complex_Words, dtype: float64

In [81]:
# Calculate the Fog Index for each row
data['Fog_Index']=0.4*(data['Average_Sentence_Length'] + data['Percentage_Of_Complex_Words'])
data['Fog_Index'].head()

0    6.199459
1    7.343018
2    7.797441
3    8.318839
4    7.159899
Name: Fog_Index, dtype: float64

In [82]:
#Since the average number of words per sentence is represented by average sentence length I am using values of average sentence length.
data['Average_Number_Of_Words_Per_Sentence']=data['Average_Sentence_Length']
data['Average_Number_Of_Words_Per_Sentence'].head()

0    15.307692
1    18.074074
2    19.142857
3    20.450980
4    17.641026
Name: Average_Number_Of_Words_Per_Sentence, dtype: float64

In [83]:
#data['Number_of_words_per_sentence'] = data_3['sentence_per_row'].apply(lambda row: [len(nltk.word_tokenize(sentence)) for sentence in row])

In [84]:
#data['Number_of_words_per_sentence']

In [85]:
# List of personal pronouns
personal_pronouns=['I','we','my','ours','us']

# Function to count personal pronouns in a list of words
def count_personal_pronouns(words):

  # Join the list of words into a single string
  text=" ".join(words)

  # Create a regular expression pattern for matching personal pronouns (case-insensitive)
  pattern=re.compile(r'\b(?:' + '|'.join(re.escape(pronoun) for pronoun in personal_pronouns)+ r')\b',flags=re.IGNORECASE)

  # Find all matches of personal pronouns in the text  excluding 'US'
  pronoun_matches=[pronoun for pronoun in pattern.findall(text) if pronoun != 'US']

   # Return the count of personal pronouns
  return len(pronoun_matches)

# Apply the count_personal_pronouns function to each element in the 'Article_text' column of the 'data_4' DataFrame
data['Personal_Pronouns']=data_4['Article_text'].apply(count_personal_pronouns)
data['Personal_Pronouns'].head()


0    12
1     4
2    13
3     5
4     6
Name: Personal_Pronouns, dtype: int64

In [86]:
data.head()

,URL_ID,URL,Article_title,Article_text,Positive_Score,Negative_Score,Polarity_Score,Word_Count,Subjectivity_Score,Syllable_Per_Word,Complex_Word_Count,number_of_sentences,number_of_words,Avg_Word_Length,Average_Sentence_Length,Percentage_Of_Complex_Words,Fog_Index,Average_Number_Of_Words_Per_Sentence,Personal_Pronouns
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"[huge, development, dependence, people, techno...",27,6,0.636364,613,0.053834,"[2, 2, 4, 4, 3, 3, 2, 2, 2, 2, 4, 2, 1, 2, 2, ...",228,78,1194,4.564489,15.307692,0.190955,6.199459,15.307692,12
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"[history, ,, industrial, revolution, century, ...",52,31,0.253012,868,0.095622,"[4, 2, 4, 5, 1, 2, 4, 3, 3, 2, 2, 4, 2, 2, 2, ...",415,81,1464,5.396858,18.074074,0.283470,7.343018,18.074074,4
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","[introduction, span, decades, ,, internet, und...",36,25,0.180328,674,0.090504,"[5, 1, 2, 3, 4, 4, 5, 3, 3, 1, 1, 3, 2, 2, 3, ...",376,56,1072,5.992537,19.142857,0.350746,7.797441,19.142857,13
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"[live, ,, work, ,, communicate, unquestionably...",35,74,-0.357798,657,0.165906,"[1, 2, 1, 5, 6, 1, 2, 2, 5, 3, 3, 2, 6, 2, 3, ...",361,51,1043,5.915628,20.450980,0.346117,8.318839,20.450980,5
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,"[poised, witness, continued, revolution, world...",21,9,0.400000,413,0.072639,"[2, 2, 2, 3, 5, 1, 5, 2, 4, 2, 1, 1, 2, 3, 2, ...",178,39,688,5.386628,17.641026,0.258721,7.159899,17.641026,6


In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   URL_ID                                98 non-null     object 
 1   URL                                   98 non-null     object 
 2   Article_title                         98 non-null     object 
 3   Article_text                          98 non-null     object 
 4   Positive_Score                        98 non-null     int64  
 5   Negative_Score                        98 non-null     int64  
 6   Polarity_Score                        98 non-null     float64
 7   Word_Count                            98 non-null     int64  
 8   Subjectivity_Score                    98 non-null     float64
 9   Syllable_Per_Word                     98 non-null     object 
 10  Complex_Word_Count                    98 non-null     int64  
 11  number_of_sentences  

In [88]:
data.drop(columns=['Article_title','Article_text','number_of_sentences','number_of_words'],inplace=True)

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   URL_ID                                98 non-null     object 
 1   URL                                   98 non-null     object 
 2   Positive_Score                        98 non-null     int64  
 3   Negative_Score                        98 non-null     int64  
 4   Polarity_Score                        98 non-null     float64
 5   Word_Count                            98 non-null     int64  
 6   Subjectivity_Score                    98 non-null     float64
 7   Syllable_Per_Word                     98 non-null     object 
 8   Complex_Word_Count                    98 non-null     int64  
 9   Avg_Word_Length                       98 non-null     float64
 10  Average_Sentence_Length               98 non-null     float64
 11  Percentage_Of_Complex

In [90]:
data.to_excel('Article_Text_Analysis_Output.xlsx',index=False)